### EXPLORE

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# To ignore warnings for clean output
import warnings
warnings.filterwarnings("ignore")

# Set visualization style
sns.set(style="whitegrid")

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Data/AA_UseCase_Data.zip")
df.columns


In [ ]:
# Renaming columns to make them more readable and intuitive
df.rename(columns={
    'step': 'time_step',
    'action': 'transaction_type',
    'amount': 'transaction_amount',
    'nameOrig': 'origin_account',
    'oldBalanceOrig': 'origin_old_balance',
    'newBalanceOrig': 'origin_new_balance',
    'nameDest': 'destination_account',
    'oldBalanceDest': 'destination_old_balance',
    'newBalanceDest': 'destination_new_balance',
    'isFraud': 'is_fraud',
    'isFlaggedFraud': 'is_flagged_fraud'
}, inplace=True)

# Preview the updated column names
df.head()


,Id,time_step,transaction_type,transaction_amount,origin_account,origin_old_balance,origin_new_balance,destination_account,destination_old_balance,destination_new_balance,is_fraud,is_flagged_fraud
0,0,0,TRANSFER,10.00,C0198526315,30112.00,30102.00,C4653045645,21927.84,21937.84,0,0
1,1,1,CASH_IN,104925.19,C9864462944,42.23,104967.41,C7853342674,0.00,0.00,0,0
2,2,1,CASH_IN,37383.17,C1474610910,49.83,37433.01,C3584357969,0.00,0.00,0,0
3,3,1,CASH_IN,131908.49,C1474610910,37433.01,169341.49,C4996589500,0.00,0.00,0,0
4,4,1,CASH_IN,127105.65,C1474610910,169341.49,296447.14,C5989915138,0.00,0.00,0,0


In [ ]:
# Uncomment if you don't have these libraries
#%pip install skrub xgboost

In [ ]:
from skrub import TableReport

TableReport(df)

,,,,,,,,,,,,


In [ ]:
df.drop('is_flagged_fraud', axis=1, inplace=True) # they are all zeros

In [ ]:
# Count of fraudulent vs non-fraudulent transactions
fraud_counts = df['is_fraud'].value_counts()

print("Transaction Class Distribution:")
print(fraud_counts)


Transaction Class Distribution:
is_fraud
0    2372627
1        178
Name: count, dtype: int64


In [ ]:
# Filter only the fraudulent transactions
fraud_df = df[df['is_fraud'] == 1]

# Check how many fraudulent transactions exist for each type
fraud_type_counts = fraud_df['transaction_type'].value_counts()

print("Fraudulent Transactions by Type:")
print(fraud_type_counts)

del fraud_df # memory efficiency

Fraudulent Transactions by Type:
transaction_type
CASH_OUT    94
TRANSFER    84
Name: count, dtype: int64


### FEATURE ENGINEERING

## Feature Engineering Açıklamaları

Aşağıda, veri seti üzerinde gerçekleştirdiğim özellik mühendisliği sürecinde oluşturulan tüm yeni feature'ların açıklamaları, hesaplama yöntemleri ve bu feature'ların dolandırıcılık tespitine nasıl katkı sağlayabileceği özetlenmiştir.

---

## 🧮 1. `balance_diff`
- **Açıklama:** İşlem tutarının, kaynak hesabın mevcut bakiyesinden çıkarılması sonucu elde edilen fark.
- **Hesaplama:** `balance_diff = oldBalanceOrig - amount`
- **Motivasyon:** Kaynak hesapta yeterli bakiye yokken yapılan işlemler şüpheli olabilir.

---

## 🧮 2. `dest_balance_diff`
- **Açıklama:** İşlem sonrası alıcı hesabın bakiyesinin işlem tutarından farkı.
- **Hesaplama:** `dest_balance_diff = newBalanceDest - amount`
- **Motivasyon:** Bazı sahte işlemlerde alıcı hesabın bakiyesinde mantıksız değişiklikler görülebilir.

---

## 📈 3. `success_rate`
- **Açıklama:** Her bir kaynak hesabın yaptığı başarılı işlemlerin oranı.
- **Hesaplama:** `success_rate = successful_tx_count / total_tx_count`
- **Motivasyon:** Başarısız işlem oranı yüksek olan kullanıcılar şüpheli davranışlar sergiliyor olabilir.

---

## 🕵️ 4. `dest_fraud_history`
- **Açıklama:** Alıcı hesabın daha önce dolandırıcılığa karışıp karışmadığını belirtir.
- **Hesaplama:** Hesap geçmişinde `isFraud == 1` varsa `1`, yoksa `0`.
- **Motivasyon:** Daha önce şüpheli olan hesapların yeniden dolandırıcılıkta yer alma olasılığı yüksektir.

---

## 📊 5. `amount_anomaly`
- **Açıklama:** İşlem tutarının, kullanıcının önceki işlem ortalamasından sapma miktarı (z-score).
- **Hesaplama:** `amount_anomaly = (amount - mean_user_amount) / std_user_amount`
- **Motivasyon:** Anormal derecede büyük veya küçük tutarlı işlemler potansiyel risk taşıyabilir.

---

## 🆕 6. `is_first_tx`
- **Açıklama:** Kullanıcının sistemdeki ilk işlemi olup olmadığını gösteren ikili değişken.
- **Hesaplama:** Her kullanıcı için sıralanmış işlemler içinde ilk olanlara `1`, diğerlerine `0` verilir.
- **Motivasyon:** Yeni açılmış hesaplar sahtekarlık amacıyla açılmış olabilir.

---

## ⏱️ 7. `time_since_last_tx`
- **Açıklama:** Kaynak hesabın önceki işleminden bu yana geçen zaman (saniye).
- **Hesaplama:** `time_since_last_tx = current_tx_time - previous_tx_time`
- **Motivasyon:** Çok kısa süre içinde yapılan ardışık işlemler şüpheli olabilir.

---

## 🚨 8. `suspicious_account`
- **Açıklama:** Önceden belirlenen kurallara göre şüpheli olarak işaretlenen hesaplar.
- **Hesaplama:** Belirli eşiklere dayalı olarak manuel veya kural tabanlı sınıflandırma (`e.g. success_rate < 0.5 && amount_anomaly > 2`)
- **Motivasyon:** Önceden anormal davranışlar sergileyen hesaplar risklidir.

---

## 🔄 9. `transfer_cashout_pair`
- **Açıklama:** Aynı miktarda önce transfer sonra nakit çekme işleminin olup olmadığını gösterir.
- **Hesaplama:** Aynı kullanıcıdan ardışık olarak yapılan TRANSFER ve CASH_OUT işlemlerinin zaman ve miktar bazlı eşleştirilmesi.
- **Motivasyon:** Bu pattern, paranın bir hesaba aktarıldıktan hemen sonra çekildiği sahtekarlık senaryolarında yaygındır.

---

## ⏳ 10. `tx_frequency_last_hour`
- **Açıklama:** Son 1 saat içinde aynı kullanıcının yaptığı işlem sayısı.
- **Hesaplama:** Zaman penceresiyle hareketli sayım (`rolling window`)
- **Motivasyon:** Kısa sürede çok işlem yapan hesaplar dikkat çeker.

---

## 🧍 11. `avg_amount_per_user`
- **Açıklama:** Kullanıcının ortalama işlem tutarı.
- **Hesaplama:** Kullanıcının yaptığı tüm işlemlerin `amount` ortalaması.
- **Motivasyon:** Kullanıcının kendi tipik işlem hacminin dışına çıkması bir sinyal olabilir.

---

## 🔗 12. `num_unique_destinations`
- **Açıklama:** Her kullanıcının işlem yaptığı benzersiz alıcı sayısı.
- **Hesaplama:** `n_distinct(destination account) per source user`
- **Motivasyon:** Çok sayıda farklı hesaba para göndermek yaygın dolandırıcılık taktiklerinden biridir.

---

## 🧭 13. `avg_time_between_txs`
- **Açıklama:** Kullanıcının iki işlem arasında geçen ortalama süresi.
- **Hesaplama:** Tüm işlemler arası zaman farklarının ortalaması.
- **Motivasyon:** Düzenli aralıklarla işlem yapan kullanıcılar daha normal davranışlar sergiler.

---

## 🔄 14. `rolling_avg_amount`
- **Açıklama:** Belirli bir zaman penceresi içinde işlem tutarlarının hareketli ortalaması.
- **Hesaplama:** `rolling(mean(amount), window = n)`
- **Motivasyon:** Zamana bağlı tutar değişimleri incelenebilir.

---


# Model Eğitim Süreci


## 1. Veri Hazırlama ve Önişleme

Eğitim süreci `main.py` dosyasında başlıyor. İlk adımlar:

- Veri setini yükleme ve örnekleme
- Özellik mühendisliği uygulama
- Veri setini eğitim/test olarak bölme
- SMOTE kullanarak veri dengesizliğini giderme

```python
# Veri yükleme
df = pd.read_csv(args.data_path)

# Özellik mühendisliği
df_processed = engineer_all_features(df)

# Eğitim/test veri setlerini ayırma
X_train, X_test, y_train, y_test = prepare_data(df_processed)

# SMOTE uygulama
X_train_res, y_train_res = apply_smote(X_train, y_train)
```

## 2. Model Seçimi ve Parametre Ayarları

Kullanıcı tarafından seçilen model tipine göre (XGBoost, LightGBM veya Random Forest) model ve hiperparametre ızgarası oluşturulur.

```python
model, param_grid = create_model_and_params(args.model_type, args.fast_mode)
```

Her bir model tipi için farklı parametreler belirlenir:

### XGBoost
```python
model = XGBClassifier(
    tree_method='gpu_hist',
    eval_metric='aucpr',
    device='cuda',
)

param_grid = {
    'max_depth': [3, 5, 7],
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    # Diğer parametreler...
}
```

### LightGBM
```python
model = lgbm.LGBMClassifier(
    objective='binary',
    metric='auc',
    device='gpu',
    verbose=-1
)

param_grid = {
    'num_leaves': [31, 63, 127],
    'n_estimators': [100, 200, 300],
    # Diğer parametreler...
}
```

### Random Forest
```python
model = RandomForestClassifier(
    n_jobs=-1,
    random_state=42,
    class_weight='balanced'
)

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30, None],
    # Diğer parametreler...
}
```

## 3. Model Eğitimi ve Hiperparametre Optimizasyonu

Model eğitimi `train_model_with_timer` fonksiyonu kullanılarak gerçekleştirilir. Bu fonksiyon `model_utils.py` dosyasında tanımlanmıştır:

```python
def train_model_with_timer(model, param_grid, model_name, X_train, y_train):
    print(f"\n🔹 {model_name} - Training model...\n")
    start_time = time.time()
    
    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        cv=3,               # 3 katlı çapraz doğrulama
        scoring='recall',   # Geri çağırma metriği kullanılıyor
        n_jobs=-1,          # Tüm CPU çekirdeklerini kullan
        verbose=0
    )
    
    grid_search.fit(X_train, y_train)
    
    end_time = time.time()
    print(f"\n✅ Training complete! Total time: {end_time - start_time:.2f} seconds")
    print(f"Best parameters: {grid_search.best_params_}\n")
    
    return grid_search
```

Bu fonksiyon:
1. Eğitim süresini ölçer
2. GridSearchCV kullanarak en iyi parametreleri bulmak için kapsamlı bir arama yapar
3. Çapraz doğrulama ile modelin genelleştirme yeteneğini değerlendirir
4. 'recall' (geri çağırma) metriğini optimize eder - dolandırıcılık tespitinde, yanlış negatifleri azaltmak kritik olduğundan bu önemlidir
5. Eğitim süresi ve en iyi parametreler hakkında bilgi sağlar

## 4. En İyi Eşik Değerinin Bulunması ve Öngörüler

Eğitim tamamlandıktan sonra, model test verisi üzerinde değerlendirilir:

```python
if hasattr(trained_model, 'predict_proba'):
    y_test_proba = trained_model.predict_proba(X_test)[:, 1]
    # Maliyet analizine dayalı en iyi eşik değerini bulma
    print("💰 Finding optimal classification threshold...")
    opt_threshold = find_optimal_threshold(y_test, y_test_proba)
    y_test_pred = (y_test_proba >= opt_threshold).astype(int)
else:
    y_test_pred = trained_model.predict(X_test)
```

`find_optimal_threshold` fonksiyonu (`evaluation_utils.py` içinde), yanlış pozitif ve yanlış negatif maliyetlerini dikkate alarak en iyi sınıflandırma eşiğini belirler:

```python
def find_optimal_threshold(y_true, y_proba, cost_fn=1000, cost_fp=50):
    """
    Maliyet değerlendirmelerine dayalı sınıflandırma için en iyi eşik değeri bulur.
    
    Args:
        y_true: Gerçek hedef değerleri
        y_proba: Tahmin olasılıkları
        cost_fn: Yanlış negatif maliyeti (kaçırılan dolandırıcılık)
        cost_fp: Yanlış pozitif maliyeti (yanlış alarm)
        
    Returns:
        float: En iyi eşik değeri
    """
    # Değişkenleri başlat
    thresholds = np.linspace(0.01, 0.99, 99)
    costs = []
    
    # Her eşik için maliyeti hesapla
    for thresh in thresholds:
        y_pred = (y_proba >= thresh).astype(int)
        
        # Karışıklık matrisi değerlerini hesapla
        tn = np.sum((y_true == 0) & (y_pred == 0))
        fp = np.sum((y_true == 0) & (y_pred == 1))
        fn = np.sum((y_true == 1) & (y_pred == 0))
        tp = np.sum((y_true == 1) & (y_pred == 1))
        
        # Toplam maliyeti hesapla
        total_cost = (fn * cost_fn) + (fp * cost_fp)
        costs.append(total_cost)
    
    # En düşük maliyete sahip eşiği bul
    optimal_idx = np.argmin(costs)
    optimal_threshold = thresholds[optimal_idx]
    
    # Maliyete karşı eşik değerini grafikle
    plt.figure(figsize=(10, 6))
    plt.plot(thresholds, costs)
    plt.axvline(x=optimal_threshold, color='r', linestyle='--')
    plt.text(optimal_threshold+0.02, np.min(costs), f'Threshold = {optimal_threshold:.2f}')
    plt.xlabel('Threshold')
    plt.ylabel('Total Cost')
    plt.title('Cost vs. Threshold')
    plt.grid(True)
    plt.savefig('optimal_threshold.png')
    plt.close()
    
    print(f"\nOptimal threshold analysis saved as 'optimal_threshold.png'")
    print(f"Optimal threshold: {optimal_threshold:.4f}")
    
    return optimal_threshold
```

Bu fonksiyon şunları yapar:
- Farklı eşik değerleri için toplam maliyet hesaplar
- Yanlış negatifler (missed fraud) ve yanlış pozitiflere (false alarms) farklı maliyetler atar
- En düşük toplam maliyete sahip eşik değerini seçer
- Sonuçları görselleştirir

## 5. Model Değerlendirmesi

Son olarak, model `evaluate_model` fonksiyonu ile kapsamlı olarak değerlendirilir:

```python
metrics = evaluate_model(
    y_train_res, y_train_pred,
    y_test, y_test_pred,
    args.model_type,
    transaction_amounts=X_test['transaction_amount'],
    model=trained_model.best_estimator_ if hasattr(trained_model, 'best_estimator_') else trained_model,
    X_test=X_test
)
```

Bu fonksiyon (`evaluation_utils.py` içinde):
- Doğruluk, kesinlik, geri çağırma, F1 skoru gibi temel metrikleri hesaplar
- ROC eğrisi ve PR eğrisi gibi değerlendirme eğrilerini çizer
- Karışıklık matrisini oluşturur
- Özellik önem grafiğini gösterir
- Para birimiyle ilgili etki analizini yapar (tespit edilen/kaçırılan dolandırıcılık tutarları)
- Yanlış sınıflandırmaları analiz eder

Son olarak, eğitilen model ve değerlendirme metrikleri `output_dir` dizinine kaydedilir.

```python
# Modeli kaydetme
model_file = f"{args.output_dir}/{args.model_type}_model.pkl"
save_model(trained_model, model_file)

# Metrikleri kaydetme
with open(f"{args.output_dir}/{args.model_type}_metrics.json", 'w') as f:
    json.dump(metrics, f, indent=4)
```

## Önemli Noktalar

1. **Veri Dengeleme**: SMOTE algoritması, azınlık sınıfından (dolandırıcılık örnekleri) sentetik örnekler oluşturarak veri setini dengeler.

2. **GridSearchCV**: Önceden belirlenmiş bir parametre ızgarasından en iyi hiperparametreleri bulmak için çapraz doğrulama yöntemi kullanılır.

3. **Maliyet-duyarlı Değerlendirme**: Modelin değerlendirilmesinde, kaçırılan dolandırıcılık ve yanlış alarmların farklı maliyetleri hesaba katılır.

4. **Finansal Etki Analizi**: Değerlendirme, basit doğruluk metriklerinin ötesine geçerek, modelin gerçek dünya finansal etkisini inceler.

Bu yaklaşım, özellikle dengesiz veri seti ve asimetrik maliyetlerle başa çıkmak için optimize edilmiş güçlü bir dolandırıcılık tespit modeli oluşturur.

# Execution of main.py ve Sonuçlar



In [5]:
!python3 main.py --data_path "/content/drive/MyDrive/Data/AA_UseCase_Data.zip" \
                 --model_type "xgboost" \
                 --output_dir "/content/output" \
                 --fast_mode

📊 Loading and preprocessing data...
🔧 Engineering features...
📈 Performing exploratory analysis...
EDA visualizations saved to /content/output/eda/
🔪 Splitting data into train/test sets...
⚖️ Applying SMOTE to balance training data...
🤖 Creating xgboost model...
🏋️‍♀️ Training model with hyperparameter tuning...

🔹 xgboost - Training model...

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [13:33:29] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [13:33:29] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(s

In [2]:
# zip and then save on drive
#!zip -r case_study_garanti.zip /content -x /content/drive/**\*
#!cp case_study_garanti.zip /content/drive/MyDrive/

# **Detailed Feature Documentation**


## Core Transaction Features

| Feature | Description | Calculation | Fraud Detection Significance |
|---------|-------------|-------------|------------------------------|
| `Id` | Unique identifier for each transaction | Assigned by system | Used for tracking and reference only |
| `time_step` | Time point when transaction occurred | System timestamp | Helps identify suspicious timing patterns |
| `transaction_type` | Type of transaction | Categories like TRANSFER, CASH_IN, PAYMENT, CASH_OUT, DEBIT | Different transaction types have varying fraud risk levels |
| `transaction_amount` | Amount of money involved | Monetary value | Fraudulent transactions often have unusual amounts |
| `origin_account` | Account initiating the transaction | Account identifier | Helps track account behavior and history |
| `origin_old_balance` | Balance before transaction | Previous account balance | Used to verify transaction validity |
| `origin_new_balance` | Balance after transaction | Updated account balance | Used to detect inconsistencies in balance changes |
| `destination_account` | Receiving account | Account identifier | Helps identify suspicious recipient patterns |
| `destination_old_balance` | Destination balance before transaction | Previous balance | Used to verify transaction consistency |
| `destination_new_balance` | Destination balance after transaction | Updated balance | Used to detect anomalies in receiving accounts |
| `is_fraud` | Target variable indicating fraud | Binary label (0/1) | The outcome variable we're trying to predict |
| `is_flagged_fraud` | System-flagged fraud indicator | Binary flag (0/1) | Can be used as a baseline comparison |

## Account Reliability Features

| Feature | Description | Calculation | Fraud Detection Significance |
|---------|-------------|-------------|------------------------------|
| `success_rate` | Proportion of non-fraudulent transactions for origin account | `1 - mean(is_fraud)` grouped by origin_account | Accounts with lower success rates are higher risk |
| `dest_success_rate` | Proportion of non-fraudulent transactions for destination account | `1 - mean(is_fraud)` grouped by destination_account | Destination accounts with low success rates may be fraudulent sinks |
| `suspicious_account` | Flag for accounts with low success rates | `(success_rate <= 0.95).astype(int)` | Directly indicates potentially risky accounts |

## Balance Discrepancy Features

| Feature | Description | Calculation | Fraud Detection Significance |
|---------|-------------|-------------|------------------------------|
| `balance_diff_origin` | Change in origin account balance | `origin_old_balance - origin_new_balance` | Should match transaction amount if legitimate |
| `balance_diff_dest` | Change in destination account balance | `destination_old_balance - destination_new_balance` | Should be inverse of transaction amount if legitimate |
| `expected_origin_balance_diff` | Expected change in origin balance | `transaction_amount` | Used to verify balance consistency |
| `expected_dest_balance_diff` | Expected change in destination balance | `-transaction_amount` | Used to verify balance consistency |
| `origin_balance_mismatch` | Flag for inconsistent origin balance changes | `abs(balance_diff_origin - expected_origin_balance_diff) > 0.01` | Inconsistencies can indicate fraud |
| `dest_balance_mismatch` | Flag for inconsistent destination balance changes | `abs(balance_diff_dest - expected_dest_balance_diff) > 0.01` | Inconsistencies can indicate fraud |
| `amount_to_balance_ratio` | Transaction amount as proportion of origin balance | `transaction_amount / origin_old_balance` (with zero-handling) | Unusually high ratios may indicate account draining |
| `zero_balance_after_tx` | Flag for transactions that empty accounts | `(origin_new_balance == 0).astype(int)` | Account emptying is often associated with fraud |
| `account_emptying` | Flag for near-empty accounts after transaction | `(origin_new_balance < origin_old_balance * 0.1).astype(int)` | Significant balance reduction is suspicious |

## Amount Anomaly Features

| Feature | Description | Calculation | Fraud Detection Significance |
|---------|-------------|-------------|------------------------------|
| `amount_Z` | Z-score of transaction amount | `(transaction_amount - mean) / std` | Identifies unusually large or small transactions |
| `amount_anomaly` | Flag for anomalous amounts | `(amount_Z > 3)` | Directly flags suspicious amounts |
| `transfer_amount_Z` | Z-score for TRANSFER transactions | Z-score calculated within TRANSFER transaction type | Identifies unusual transfer amounts |
| `transfer_amount_anomaly` | Flag for anomalous TRANSFER amounts | `(transfer_amount_Z > 3)` | Specific to TRANSFER type anomalies |
| `cashin_amount_Z` | Z-score for CASH_IN transactions | Z-score calculated within CASH_IN transaction type | Identifies unusual cash deposit amounts |
| `cashin_amount_anomaly` | Flag for anomalous CASH_IN amounts | `(cashin_amount_Z > 3)` | Specific to CASH_IN type anomalies |
| `payment_amount_Z` | Z-score for PAYMENT transactions | Z-score calculated within PAYMENT transaction type | Identifies unusual payment amounts |
| `payment_amount_anomaly` | Flag for anomalous PAYMENT amounts | `(payment_amount_Z > 3)` | Specific to PAYMENT type anomalies |
| `cashout_amount_Z` | Z-score for CASH_OUT transactions | Z-score calculated within CASH_OUT transaction type | Identifies unusual withdrawal amounts |
| `cashout_amount_anomaly` | Flag for anomalous CASH_OUT amounts | `(cashout_amount_Z > 3)` | Specific to CASH_OUT type anomalies |
| `debit_amount_Z` | Z-score for DEBIT transactions | Z-score calculated within DEBIT transaction type | Identifies unusual debit amounts |
| `debit_amount_anomaly` | Flag for anomalous DEBIT amounts | `(debit_amount_Z > 3)` | Specific to DEBIT type anomalies |
| `high_transaction` | Flags transactions with large amounts | Based on 95th percentile threshold | Large amounts may indicate fraud attempts |

## Transaction History Features

| Feature | Description | Calculation | Fraud Detection Significance |
|---------|-------------|-------------|------------------------------|
| `is_first_tx` | Flag for first transaction from account | Based on absence of previous transactions | New accounts are higher risk for fraud |
| `time_since_last_tx` | Time steps since previous transaction | Current timestamp minus previous timestamp | Unusual timing patterns may indicate fraud |
| `tx_count_last_5_steps` | Number of transactions in past 5 time steps | Rolling count with window=5 | Rapid transaction frequency can signal fraud |
| `tx_count_last_10_steps` | Number of transactions in past 10 time steps | Rolling count with window=10 | Medium-term transaction velocity |
| `tx_count_last_20_steps` | Number of transactions in past 20 time steps | Rolling count with window=20 | Longer-term transaction patterns |
| `time_hour` | Hour of day when transaction occurred | `time_step % 24` | Time-of-day patterns can relate to fraud |
| `is_night` | Flag for nighttime transactions | `((time_hour >= 22) \| (time_hour <= 5)).astype(int)` | Transactions at unusual hours can be suspicious |
| `is_weekend` | Flag for weekend transactions | `(time_step % 7 >= 5).astype(int)` | Weekend patterns may differ for fraud |

## Account Aggregation Features

| Feature | Description | Calculation | Fraud Detection Significance |
|---------|-------------|-------------|------------------------------|
| `origin_tx_count` | Total transactions from origin account | Count of transactions by origin_account | Activity level can indicate risk |
| `origin_mean_amount` | Average transaction amount from account | Mean of transaction_amount grouped by origin_account | Deviation from usual patterns can signal fraud |
| `origin_max_amount` | Maximum transaction amount from account | Max of transaction_amount grouped by origin_account | Context for whether current amount is unusual |
| `origin_std_amount` | Standard deviation of amounts from account | Std of transaction_amount grouped by origin_account | Indicates typical variation in transaction amounts |
| `origin_amount_deviation` | How far current amount deviates from mean | `abs(transaction_amount - origin_mean_amount)` | Large deviations may be suspicious |
| `origin_amount_deviation_ratio` | Normalized deviation from typical amounts | `origin_amount_deviation / (origin_std_amount + epsilon)` | Adjusts for accounts with naturally high variance |
| `dest_tx_count` | Total transactions to destination account | Count of transactions by destination_account | High-volume receiving accounts may be suspicious |
| `dest_mean_amount` | Average transaction amount to destination | Mean of transaction_amount grouped by destination_account | Context for destination account patterns |
| `dest_max_amount` | Maximum amount received by destination | Max of transaction_amount grouped by destination_account | Context for current transaction |
| `dest_std_amount` | Standard deviation of amounts to destination | Std of transaction_amount grouped by destination_account | Indicates typical variation in amounts received |

## Risk Assessment Features

| Feature | Description | Calculation | Fraud Detection Significance |
|---------|-------------|-------------|------------------------------|
| `high_risk_transaction` | Flag for risky transaction types | Based on transaction_type being TRANSFER or CASH_OUT | These types are more associated with fraud |
| `multiple_risk_factors` | Combines several risk indicators | Combines high_risk_transaction, high_transaction, and amount_anomaly | Transactions with multiple red flags are higher risk |
| `fraud_risk_score` | Overall risk score | Average of multiple binary risk factors | Summarizes overall fraud risk level |

## Network Analysis Features

| Feature | Description | Calculation | Fraud Detection Significance |
|---------|-------------|-------------|------------------------------|
| `account_pair` | Unique identifier for transaction pair | `origin_account + '_' + destination_account` | Identifies specific account relationships |
| `pair_tx_count` | Number of transactions between account pair | Count grouped by account_pair | Unusual relationships or patterns between accounts |
| `origin_partner_count` | Number of unique destinations for origin | Count of unique destination_account grouped by origin_account | Accounts dealing with many partners may be suspicious |
| `dest_partner_count` | Number of unique origins for destination | Count of unique origin_account grouped by destination_account | Accounts receiving from many sources may be suspicious |
| `transfer_cashout_pair` | Flag for accounts in suspicious patterns | Based on intersection of transfer destinations and cashout origins | Classic money laundering pattern |

## Transaction Velocity Features

| Feature | Description | Calculation | Fraud Detection Significance |
|---------|-------------|-------------|------------------------------|
| `tx_velocity_increase` | Measure of rapid transaction increase | `tx_velocity_10 / (tx_velocity_60 + epsilon)` | Sudden bursts of activity may indicate fraud |
| `unusual_velocity` | Flag for abnormal transaction velocity | `(tx_velocity_increase > 3).astype(int)` | Directly flags suspicious velocity increases |

## Transaction Sequence Features

| Feature | Description | Calculation | Fraud Detection Significance |
|---------|-------------|-------------|------------------------------|
| `rapid_succession` | Flag for quickly successive transactions | `(time_gap < 2).astype(int)` | Multiple quick transactions may indicate fraud |
| `large_amount_increase` | Flag for large increases in transaction amount | `(amount_change_pct > 5).astype(int)` | Sudden increases in amounts can be suspicious |
| `small_then_large` | Pattern of small transaction followed by large one | Based on percentile changes | Testing with small amounts then large fraud is common |
| `cash_in_then_transfer` | Flag for CASH_IN followed by TRANSFER | Pattern matching on consecutive transaction types | Common money laundering pattern |
| `transfer_then_cash_out` | Flag for TRANSFER followed by CASH_OUT | Pattern matching on consecutive transaction types | Common money laundering pattern |

## Account Pattern Features

| Feature | Description | Calculation | Fraud Detection Significance |
|---------|-------------|-------------|------------------------------|
| `origin_fraud_history` | Flag for accounts with fraud history | Based on accounts involved in past fraud | Past behavior is predictive of future behavior |
| `dest_fraud_history` | Flag for destination accounts with fraud history | Based on destination accounts involved in past fraud | Identifies known suspicious recipients |
| `in_transaction_cycle` | Flag for accounts in transaction cycles | Based on network cycle detection | Cyclic money movement can indicate laundering |
| `bidirectional_tx` | Flag for accounts with bidirectional flow | Accounts that both send and receive from each other | Money circulation can indicate fraud schemes |
| `new_account` | Flag for recently created accounts | `(account_age < 30).astype(int)` | New accounts are higher risk for fraud |
| `account_first_seen` | Time step when account first appeared | Min time_step for each origin_account | Used to calculate account age |

## Advanced Pattern Features

These features combine multiple signals to identify sophisticated fraud patterns:

| Feature | Description | Calculation | Fraud Detection Significance |
|---------|-------------|-------------|------------------------------|
| `high_risk_origin_type` | Flag for high-risk account type (origin) | Based on fraud rates by account prefix type | Some account types are more prone to fraud |
| `high_risk_dest_type` | Flag for high-risk account type (destination) | Based on fraud rates by account prefix type | Some recipient types are more associated with fraud |
| `origin_prefix_fraud_rate` | Fraud rate for origin account type | Mean fraud rate by account type prefix | Quantifies risk level of account type |
| `dest_prefix_fraud_rate` | Fraud rate for destination account type | Mean fraud rate by account type prefix | Quantifies risk level of recipient type |